#### Dependencies
____

In [7]:
import pandas as pd
import pickle

from sqlalchemy import create_engine
from fbprophet import Prophet

#### Load Data
____

In [25]:
dataset = '../resources/daily_household_power_consumption.csv'

master = pd.read_csv(dataset, infer_datetime_format=True, parse_dates=['datetime'] )
master.head()

,datetime,Global_active_power,Global_reactive_power,Voltage,Global_intensity,kitchen_active_energy,laundry_active_energy,water_heater_ac_active_energy,equipment_active_energy
0,2006-12-16,1209.176,34.922,93552.53,5180.8,0.0,546.0,4926.0,14680.933319
1,2006-12-17,3390.460,226.006,345725.32,14398.6,2033.0,4187.0,13341.0,36946.666732
2,2006-12-18,2203.826,161.792,347373.64,9247.2,1063.0,2621.0,14018.0,19028.433281
3,2006-12-19,1666.194,150.942,348479.01,7094.0,839.0,7602.0,6197.0,13131.900043
4,2006-12-20,2225.748,160.998,348923.61,9313.0,0.0,2648.0,14063.0,20384.800011


#### Adjust columns names
____

In [26]:
master.rename(columns = {
    'Global_active_power'           : 'global',
    'Global_reactive_power'         : 'reactive',
    'Voltage'                       : 'voltage',
    'Global_intensity'              : 'intensity',
    'kitchen_active_energy'         : 'kitchen',
    'laundry_active_energy'         : 'laundry',
    'water_heater_ac_active_energy' : 'water',
    'equipment_active_energy'       : 'equipment'
},inplace=True)

#### Export to SQLite 
____

In [27]:
disk_engine = create_engine('sqlite:///../database/energy.db')
master.to_sql(name='energy', con=disk_engine, if_exists='replace')

### Export Models
____

In [16]:
features = ['Global_active_power', 'kitchen_active_energy', 'laundry_active_energy', 'water_heater_ac_active_energy', 'equipment_active_energy']

for f in features:
    model_df = master.copy(deep=True)
    model_df = model_df[['datetime', f]]
    model_df.columns = ['ds', 'y']
    
    model = Prophet(seasonality_mode='multiplicative').fit(model_df)    
    pickle.dump(model, open('../models/' + f.split('_')[0].lower() + '.pkl','wb'))

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
